In [42]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [43]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 

In [72]:

wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [45]:

soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})

In [59]:
df=pd.read_html(str(indiatable))
df=pd.DataFrame(df[0])


print(df.head())

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


Install Beautifoulsoup4
import libraries
Make a response to bring out a status code. If is 200 is OK.
Make a DataFrame which read de html table


In [60]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [61]:
df = df.drop(df[df['Borough'] == "Not assigned"].index)

Drop "Not assigned" rows in the column 'Borough' 

In [62]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [63]:
df.groupby("Postal Code",axis=1)

In [64]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Group by Postal Code to have unique values in that column, creating a multi neighbourhood space

In [66]:
df.loc[df['Neighbourhood'] == "Not assigned"]

,Postal Code,Borough,Neighbourhood


Isn't "Not assigned" rows

In [67]:
df.loc[df['Borough'] == 'Not Assigned']

,Postal Code,Borough,Neighbourhood


In [68]:
df.shape

(103, 3)